In [1]:
from NeuralNetwork import NNetwork
from sklearn.datasets import fetch_openml
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import json
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [3]:
def train_model(model, X_train, y_train, X_val, y_val, batch_size=32, learning_rate=0.01, epochs=10, verbose=1, reg_type="L1"):
    """
    Melatih model dengan parameter yang diberikan.

    :param model: Objek dari NNetwork.
    :param X_train: Data training (numpy array, shape: (num_samples, num_features))
    :param y_train: Label training (numpy array, shape: (num_samples, num_classes))
    :param X_val: Data validasi (numpy array, shape: (num_samples, num_features))
    :param y_val: Label validasi (numpy array, shape: (num_samples, num_classes))
    :param batch_size: Jumlah sampel per batch saat training.
    :param learning_rate: Learning rate untuk gradient descent.
    :param epochs: Jumlah epoch untuk training.
    :param verbose: 0 = tanpa output, 1 = progress bar + training & validation loss.
    :return: Dictionary berisi histori training loss & validation loss tiap epoch.
    """
    history = {"train_loss": [], "val_loss": []}
    num_samples = X_train.shape[0]

    for epoch in range(epochs):
        epoch_loss = 0
        num_batches = num_samples // batch_size

        batch_iterator = tqdm(range(num_batches), desc=f"Epoch {epoch+1}/{epochs}", disable=(verbose == 0))

        for batch_idx in batch_iterator:
            start_idx = batch_idx * batch_size
            end_idx = start_idx + batch_size
            X_batch, y_batch = X_train[start_idx:end_idx], y_train[start_idx:end_idx]

            loss = model.backward_propagation(X_batch, y_batch, learning_rate, reg_type)
            epoch_loss += loss

            if verbose == 1:
                batch_iterator.set_postfix(train_loss=loss)

        train_loss = epoch_loss / num_batches
        history["train_loss"].append(train_loss)

        val_preds = model.forward_propagation(X_val)
        val_loss = np.mean((val_preds - y_val) ** 2)
        history["val_loss"].append(val_loss)

        if verbose == 1:
            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.5f}, Val Loss: {val_loss:.5f}")

    return history

In [4]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
X= X / 255
y = np.array(y).astype(int)  # Convert to integers
num_classes = 10  # since labels are from 0 to 9
y = np.eye(num_classes)[y]

# X = X[:1000]
# y = y[:1000]

Xtrain = X[:5600]
ytrain = y[:5600]
Xval = X[5600:]
yval = y[5600:]

In [ ]:
nn1 = NNetwork(4 , [784, 16, 16, 10], verbose=True)
nn1.initialize_weights(method="normal", mean=0, variance=0.1, seed=42, verbose=False)
nn2 = NNetwork(4 , [784, 16, 16, 10], verbose=True)
nn2.initialize_weights(method="normal", mean=0, variance=0.1, seed=42, verbose=False)
nn3 = NNetwork(4 , [784, 16, 16, 10], verbose=True)
nn3.initialize_weights(method="normal", mean=0, variance=0.1, seed=42, verbose=False)

In [ ]:
history1 = train_model(nn1, Xtrain, ytrain, Xval, yval, batch_size=20, learning_rate=0.01, epochs=10, verbose=1, reg_type="None")
history2 = train_model(nn2, Xtrain, ytrain, Xval, yval, batch_size=20, learning_rate=0.01, epochs=10, verbose=1, reg_type="L1")
history3 = train_model(nn3, Xtrain, ytrain, Xval, yval, batch_size=20, learning_rate=0.01, epochs=10, verbose=1, reg_type="L2")

In [9]:
def plot_training_history(history, base_dir="hasil"):
    """Plot training history and save to a dynamically created folder."""
    # Cari folder dengan angka berikutnya yang belum ada
    i = 1
    while os.path.exists(f"{base_dir}/{i}"):
        i += 1

    filename = f"training_history_{i}.png"
    folder_path = f"{base_dir}/{i}"
    os.makedirs(folder_path, exist_ok=True)

    # Simpan plot ke file di folder tersebut
    file_path = os.path.join(folder_path, filename)
    plt.figure(figsize=(10, 5))
    plt.plot(history["train_loss"], label="Training Loss", marker="o")
    plt.plot(history["val_loss"], label="Validation Loss", marker="s")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig(file_path)  # Simpan plot ke file
    plt.close()  # Tutup plot untuk menghindari masalah GUI

    # Simpan data verbose ke file teks
    verbosename = f"verbose_{i}.txt"
    verbose_file_path = os.path.join(folder_path, verbosename)
    with open(verbose_file_path, "w") as f:
        f.write(json.dumps(history, indent=4))

    print(f"Plot saved to: {file_path}")
    print(f"Verbose data saved to: {verbose_file_path}")
    return folder_path


In [ ]:
plot_training_history(history1)
plot_training_history(history2)
plot_training_history(history3)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import os
import json

# Normalisasi data
from sklearn.datasets import fetch_openml
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
X = X / 255.0
y = np.array(y).astype(int)

# One-hot encoding untuk y
num_classes = 10
y_onehot = np.eye(num_classes)[y]

# Split manual agar sesuai
Xtrain = X[:5600]
ytrain = y[:5600]
Xval = X[5600:]
yval = y[5600:]
yval_onehot = y_onehot[5600:]

def train_sklearn_model(Xtrain, ytrain, Xval, yval, alpha=0.0001, reg_type="l2", epochs=10, verbose=1):
    history = {"train_loss": [], "val_loss": []}

    model = MLPClassifier(hidden_layer_sizes=(16, 16),
                          activation='relu',
                          solver='sgd',
                          learning_rate_init=0.01,
                          max_iter=1,
                          warm_start=True,
                          batch_size=20,
                          alpha=alpha,
                          verbose=0)

    for epoch in range(epochs):
        model.fit(Xtrain, ytrain)

        # Predict untuk log loss (menggunakan probabilitas)
        train_proba = model.predict_proba(Xtrain)
        val_proba = model.predict_proba(Xval)

        train_loss = log_loss(ytrain, train_proba)
        val_loss = log_loss(yval, val_proba)

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)

        if verbose:
            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.5f}, Val Loss: {val_loss:.5f}")

    return model, history


: 

In [ ]:
model_none, history_none = train_sklearn_model(Xtrain, ytrain, Xval, yval, alpha=0.0, reg_type="none")
model_l1, history_l1 = train_sklearn_model(Xtrain, ytrain, Xval, yval, alpha=0.001, reg_type="l1")
model_l2, history_l2 = train_sklearn_model(Xtrain, ytrain, Xval, yval, alpha=0.001, reg_type="l2")

In [ ]:
def plot_training_history_sklearn(history, base_dir="hasil_sklearn"):
    i = 1
    while os.path.exists(f"{base_dir}/{i}"):
        i += 1

    filename = f"training_history_{i}.png"
    folder_path = f"{base_dir}/{i}"
    os.makedirs(folder_path, exist_ok=True)

    file_path = os.path.join(folder_path, filename)
    plt.figure(figsize=(10, 5))
    plt.plot(history["train_loss"], label="Training Loss", marker="o")
    plt.plot(history["val_loss"], label="Validation Loss", marker="s")
    plt.xlabel('Epochs')
    plt.ylabel('Log Loss')
    plt.title('Training and Validation Loss (sklearn MLPClassifier)')
    plt.legend()
    plt.grid(True)
    plt.savefig(file_path)
    plt.close()

    verbosename = f"verbose_{i}.txt"
    verbose_file_path = os.path.join(folder_path, verbosename)
    with open(verbose_file_path, "w") as f:
        f.write(json.dumps(history, indent=4))

    print(f"Plot saved to: {file_path}")
    print(f"Verbose data saved to: {verbose_file_path}")
    return folder_path

plot_training_history_sklearn(history_none)
plot_training_history_sklearn(history_l1)
plot_training_history_sklearn(history_l2)